In [ ]:
library(sf)      # simple features packages for handling vector GIS data
library(httr2)   # http-request package
library(ows4R)   # interface for OGC webservices
library(ggplot2) # stabdard plotting
library(tmap)    # sophisticated map making

The base URL for the railway network

In [ ]:
railway_wfs_url <- 'https://geoserver.geonet-mrn.de/xdatatogo/db_strecken/ows'

**Use WFSClient from ows4R**  to figure out the FeatureType(s).  
This is a shortcut through GetCapabilities.  
[Reference WFSClient](https://eblondel.github.io/ows4R/reference/WFSClient.html?q=WFSClient)

In [ ]:
WFS <- WFSClient$new(railway_wfs_url, serviceVersion = "2.0.0")
WFS$getFeatureTypes(pretty=TRUE) |> print()


Build a `GetFeature`- request

In [ ]:
parameters <- list(service = 'wfs',
                   request = 'GetFeature',
                   typeNames = 'xdatatogo:db_strecken'
                   )

req <- request(railway_wfs_url) |> req_url_query(!!!parameters)
print(req)

Get the Feature through a raw http-request **using httr2** ...

Actually perform the request and look at the **raw response**:

In [ ]:
req |> req_perform() |> resp_raw()

Read the response as **sf-object**:

In [ ]:
features <- read_sf(req$url)
print(features)

Alternatively: **Use WFSClient$GetFeatures** to get an **sf-object**:


In [ ]:
features <- WFS$getFeatures("xdatatogo:db_strecken")
print(features)

In [ ]:
ggplot(features) + geom_sf()

In [ ]:
features1 <- st_cast(features, "MULTILINESTRING")

In [ ]:
ggplot(features1) + geom_sf()

In [ ]:
st_crs(features1) <- "WGS84"

In [ ]:
print(features1)

In [ ]:
ggplot(features1) + geom_sf()

### Clip the railway network to WGS 84 (EPSG:4326) bounding box of Lower Saxony.
The BBOX is defined by its corner points: (min_long, min_lat), (max_long, max_lat)

In [ ]:
bbox <- c(xmin=6.345854, ymin=51.295232, xmax=11.598078, ymax=54.13791)


In [ ]:
strecken_ni <- st_crop(features1, bbox)

In [ ]:
ggplot(strecken_ni) + geom_sf()

In [ ]:
plot1 <- tm_shape(strecken_ni) + tm_lines(col="blue")
tmap_leaflet(plot1)

### Project to ETRS89 / UTM zone 32N (has unit meter)

In [ ]:
strecken_ni <- st_transform(strecken_ni, crs=25832)

In [ ]:
st_crs(strecken_ni)

### Put a buffer around railway network

In [ ]:
buffer_distance_m <- 2000
#buffer_distance_m <- 20

In [ ]:
strecken_ni_buffer <- st_buffer(strecken_ni, buffer_distance_m)

In [ ]:
print(strecken_ni_buffer)

In [ ]:
plot1 <- tm_shape(strecken_ni_buffer) + tm_polygons(col="blue")
tmap_leaflet(plot1)

### Load [forest dataset from WFS](https://mis.bkg.bund.de/trefferanzeige?docuuid=75C069E4-D760-49FF-BD71-5188CF81B4D9) of BKG.

- The relevant dataset "AX_Wald" is identified by the cryptic layer name `dlm250:objart_43002_f`.
- This governmental server returns geometries referenced by default in UTM32 (EPSG:25832). Thus, we can also skip additional re-projection.
- We also skip a subsequent clipping step by directly including the BBOX as spatial filter in the WFS request. Because our BBOX is specified in WGS 84 longitude/latitude, we have to explicitly provide the reference system identifier EPSG:4326 for the server to correctly interpret the BBOX coordinates. 

In [ ]:
forest_wfs_url <- 'https://sgx.geodatenzentrum.de/wfs_dlm250'

In [ ]:
forest_wfs_params <- list(
    service='wfs',
    request='GetFeature',
    typeNames='dlm250:objart_43002_f',
    bbox=paste(paste(bbox, collapse=','), ',EPSG:4326')
)
req <- request(forest_wfs_url) |> req_url_query(!!!forest_wfs_params)

In [ ]:
forest <- read_sf(req$url)
st_geometry_type(forest)[1]

### Cast to Geometrycollection

In [ ]:
forest1 <- st_cast(forest, "GEOMETRYCOLLECTION")
st_geometry_type(forest1)[1]

In [ ]:
ggplot(forest1) + geom_sf()

### Intersect railway-buff polygons with forest polygons

In [ ]:
res <- st_intersection(strecken_ni_buffer, forest1)

In [ ]:
ggplot(res) + geom_sf(linewidth=2)

In [ ]:
plot2 <- tm_shape(res) +  tm_borders(col="blue", lwd=2)
tmap_leaflet(plot2)